In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os
from fnmatch import fnmatch
from shapely.ops import unary_union
from shapely import geometry
import logging
import matplotlib.pyplot as plt

from collections import OrderedDict
%matplotlib inline

from utils import *
from test_overlap_geometry import test_overlap_geometry
from gdf_equal_id_geom import gdf_equal_id_geom
from gdf_overlap_signficant import gdf_overlap_signficant
from gdf_overlap_not_significant import gdf_overlap_not_significant
from gdf_non_equal_id import gdf_non_equal_id

In [2]:
import_folder  = 'D:/Users/Driebergen/Documents/PR/PR4019/Dissolve_shapes/export'
export_folder  = 'D:/Users/Driebergen/Documents/PR/PR4019/HDSR_TDLR/export'
current_folder = 'D:/Users/Driebergen/Documents/PR/PR4019/HDSR_TDLR/hdsr_tdlr'

In [3]:
#Find all peilgebieden shapes in subfolders
root = import_folder
pattern = "*Peil*.shp"
shapes_Peil = []
dates_Peil  = []

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, "*PEIL*.shp"):
            shapes_Peil.append(os.path.join(path, name))
            date = name.split('.shp')[0][-7:]
            dates_Peil.append(date)
            
df_shapes = pd.DataFrame()
df_shapes['shp'] = shapes_Peil
df_shapes['Date'] = dates_Peil
df_sort = df_shapes.sort_values('Date',ascending=False)
shapes_peil = df_sort['shp'].values

In [4]:
shapes_test = df_sort['shp'].values
logger = setup_custom_logger('Log_file.txt')

gdf_update_geometry = gpd.read_file(shapes_test[0]) #Initialise:2019_04 shape
gdf_update_geometry['STARTDATE'] = int('20190401')
gdf_update_geometry['ENDDATE']   = int('21000101')
gdf_update_geometry['SUFFIX']    = 0
gdf_update_geometry['Original']  = True

for idx in range(len(shapes_test)-1):
    date_base   = shapes_test[idx].split('.shp')[0][-7:]   + '01'
    date_update = shapes_test[idx+1].split('.shp')[0][-7:] + '01'
    
    gdf_base    = gdf_update_geometry
    gdf_base_rm = gdf_base[gdf_base['STARTDATE'] != int(date_base)][
        ['CODE','SUFFIX', 'STARTDATE', 'ENDDATE', 'geometry', 'Original']] # remaining geoms
    print(len(gdf_base_rm))
    gdf_base    = gdf_base[gdf_base['STARTDATE'] == int(date_base)] #Only compare geometries which are still vallid
    gdf_update  = gpd.read_file(shapes_test[idx+1])
    
    print('Comparing shape {} with shape {}'.format(date_base,date_update))
    logger.info(('Comparing shape {} with shape {}'.format(date_base,date_update)))
    logger.info(('There are {} records in shape {}'.format(len(gdf_base),date_base)))
    logger.info(('There are {} records in shape {}'.format(len(gdf_update),date_update)))
    
    col1                = find_attribuut_name(gdf_base,['GPGIDENT','CODE'])[0]
    col2                = find_attribuut_name(gdf_update,['GPGIDENT','CODE'])[0]
    
    #Search for the same ID's and calculate overlap
    same_ids                 = same_id(gdf_base,gdf_update,col_name1 = col1,col_name2 = col2)
    old_ids                  = old_id(gdf_base,gdf_update,col_name1 = col1,col_name2 = col2)
    test_geom                = test_overlap_geometry(gdf_base,gdf_update,same_ids,logger)    
    test_geom_equal          = test_geom[test_geom['Equal'] == True]
   
    test_geom_overlap        = test_geom[(test_geom['Intersect']) == True & (test_geom['Equal'] == False)]  
    geom_overl_signific      = test_geom_overlap[(test_geom_overlap['perc_base'] >= 0.8) & (test_geom_overlap['perc_update'] >= 0.8)]
    geom_overl_not_signific  = test_geom_overlap[(test_geom_overlap['perc_base'] < 0.8) | (test_geom_overlap['perc_update'] < 0.8)]

    gdf_equal_ID_GEOM        = gdf_equal_id_geom(gdf_base,test_geom_equal,date_base,date_update,logger)
    gdf_non_equal_ID         = gdf_non_equal_id(gdf_base,old_ids,date_base,date_update,logger)    
    gdf_overlap_signific     = gdf_overlap_signficant(gdf_base,gdf_update,geom_overl_signific,date_base,date_update,logger)
    gdf_overlap_not_signific = gdf_overlap_not_significant(gdf_base,gdf_update,geom_overl_not_signific,date_base,date_update,logger)
    
    gdf_update_geometry      = pd.concat([gdf_equal_ID_GEOM,gdf_non_equal_ID,gdf_overlap_signific, gdf_overlap_not_signific, gdf_base_rm])
    
    logger.info('The new dataframe has {} records'.format(len(gdf_update_geometry)))
    logger.info('-----------------------------------------------------------------------------')

0
Comparing shape 2019_0401 with shape 2019_0301
10
Comparing shape 2019_0301 with shape 2019_0201


C:\Users\Driebergen\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\base.py:29: UserWarning: GeoSeries crs mismatch: None and {'proj': 'sterea', 'lat_0': 52.15616055555555, 'lon_0': 5.38763888888889, 'k': 0.9999079, 'x_0': 155000, 'y_0': 463000, 'ellps': 'bessel', 'units': 'm', 'no_defs': True}
  other.crs))


10
Comparing shape 2019_0201 with shape 2019_0101
10
Comparing shape 2019_0101 with shape 2018_1201
10
Comparing shape 2018_1201 with shape 2018_1101
13
Comparing shape 2018_1101 with shape 2018_1001
13
Comparing shape 2018_1001 with shape 2018_0601
22
Comparing shape 2018_0601 with shape 2018_0501
26
Comparing shape 2018_0501 with shape 2018_0401
27
Comparing shape 2018_0401 with shape 2017_1101
67
Comparing shape 2017_1101 with shape 2017_0401
88
Comparing shape 2017_0401 with shape 2017_0301
98
Comparing shape 2017_0301 with shape 2017_0101
133
Comparing shape 2017_0101 with shape 2016_1201
169
Comparing shape 2016_1201 with shape 2016_1101
190
Comparing shape 2016_1101 with shape 2016_1001
249
Comparing shape 2016_1001 with shape 2016_0901
303
Comparing shape 2016_0901 with shape 2016_0801
395
Comparing shape 2016_0801 with shape 2016_0701
586
Comparing shape 2016_0701 with shape 2016_0601
586
Comparing shape 2016_0601 with shape 2016_0501
592
Comparing shape 2016_0501 with shape 2

C:\Users\Driebergen\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\base.py:29: UserWarning: GeoSeries crs mismatch: None and {}
  other.crs))


In [ ]:
gdf_update_geometry_string = gdf_update_geometry.copy()
gdf_update_geometry_string.STARTDATE = pd.to_datetime(gdf_update_geometry_string.STARTDATE, format='%Y%m%d', errors='coerce').astype(str)
gdf_update_geometry_string.ENDDATE = pd.to_datetime(gdf_update_geometry_string.ENDDATE, format='%Y%m%d', errors='coerce').astype(str)
del gdf_update_geometry_string['Original']
os.chdir(export_folder)
gdf_update_geometry_string.to_file('TDP_poly_2019_04_2010.shp')
os.chdir(current_folder)

In [ ]:
#Next, find polyons with the same geometry
shapes_test     = df_sort['shp'].values

for idx in range(len(shapes_test)-1):
    print(shapes_test[idx])
    date_base   = shapes_test[idx].split('.shp')[0][-7:]   + '01'
    date_update = shapes_test[idx+1].split('.shp')[0][-7:] + '01'
    
    gdf_base    = gpd.read_file(shapes_test[idx])
    gdf_update  = gpd.read_file(shapes_test[idx+1])    
    col1                = find_attribuut_name(gdf_base,['GPGIDENT','CODE'])[0]
    col2                = find_attribuut_name(gdf_update,['GPGIDENT','CODE'])[0]
    same_ids            = same_id(gdf_base,gdf_update,col_name1 = col1,col_name2 = col2)
    
    unique_base = unique_ids(gdf_base[col1].values,same_ids)
    unique_update = unique_ids(gdf_base[col2].values,same_ids)
    
    for id_update in unique_update:
        geom_update = gdf_update[gdf_update[col2] == id_update]['geometry']
        for id_base in unique_base:
            geom_base = gdf_base[gdf_base[col1] == id_base]['geometry']
            ovl = geom_update.overlaps(geom_base)
            if sum(ovl) == len(ovl):
                print('overlaps')
            #print(geom_update.overlaps(geom_base))
                geom_int      = geom_base.intersection(geom_update)
                opp_int       = geom_int.area
                opp_base      = geom_base.area
                opp_update    = geom_update.area
                perc_base     = opp_int / opp_base
                perc_update   = opp_int / opp_update
